In [ ]:
# Standard Library
import os
import time
import gc

# Scientific Stack
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Audio & Image Processing
import librosa
from skimage.transform import resize  # Pastikan scikit-image terinstal

# Utils & Parallel Processing
from tqdm import tqdm
from tqdm.auto import tqdm as tqdm_auto
from joblib import Parallel, delayed

# Machine Learning (sklearn)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

# Deep Learning (TensorFlow / Keras)
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    TimeDistributed, BatchNormalization, LSTM
)
from tensorflow.keras.callbacks import EarlyStopping

# Joblib
import joblib


## Waktu Ekstraksi

In [ ]:
data_path = pd.read_csv("../data/processed/data_asv_spoof.csv")
data_path

,speaker_id,filename,attack_id,class_name,filepath,target,subset
0,LA_0085,LA_T_3891170,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,training
1,LA_0097,LA_T_2948101,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,training
2,LA_0084,LA_T_8919107,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,training
3,LA_0081,LA_T_6650735,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,training
4,LA_0086,LA_T_5362691,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,training
...,...,...,...,...,...,...,...
37448,LA_0013,LA_E_7178035,A19,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,testing
37449,LA_0013,LA_E_5285610,A19,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,testing
37450,LA_0007,LA_E_5912220,A19,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,testing
37451,LA_0015,LA_E_5036389,A19,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,testing


In [3]:
data_sample_path = data_path[data_path["subset"].isin(["training", "validation"])].sample(1000, random_state=42)
data_sample_path.head(5)

,speaker_id,filename,attack_id,class_name,filepath,target,subset
4755,LA_0090,LA_T_9272004,A03,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,training
8909,LA_0104,LA_D_5261984,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,validation
5794,LA_0087,LA_T_5217553,A04,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,training
4046,LA_0081,LA_T_6276812,A02,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,training
4347,LA_0086,LA_T_7668839,A03,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,training


In [ ]:
def time_single_file_extraction(filepath):
    """
    Fungsi ini menjalankan proses ekstraksi fitur untuk satu file
    dan HANYA mengembalikan waktu yang dibutuhkan (dalam detik).
    """
    start_time = time.perf_counter()
    try:
        # Proses ekstraksi tetap dijalankan untuk mendapatkan waktu yang akurat
        y, sr = librosa.load(filepath, sr=None, mono=True)
        S = np.abs(librosa.stft(y))

        _ = librosa.feature.chroma_stft(S=S, sr=sr)
        _ = librosa.feature.rms(S=S)
        _ = librosa.feature.spectral_centroid(S=S, sr=sr)
        _ = librosa.feature.spectral_bandwidth(S=S, sr=sr)
        _ = librosa.feature.spectral_rolloff(S=S, sr=sr)
        _ = librosa.feature.zero_crossing_rate(y)
        _ = librosa.feature.mfcc(S=librosa.power_to_db(S), sr=sr, n_mfcc=20)
        
        # Hitung selisih waktu
        elapsed_time = time.perf_counter() - start_time
        return elapsed_time

    except Exception as e:
        # Jika ada file yang error, waktu tidak dicatat
        # print(f"Error processing {filepath}: {e}")
        return None

# List untuk menampung catatan waktu dari setiap file
extraction_times = []

print("Memulai pengukuran waktu ekstraksi file satu per satu...")
# Loop melalui setiap file di data sample
for filepath in tqdm(data_sample_path["filepath"], total=len(data_sample_path)):
    
    # Panggil fungsi untuk mengukur waktu
    time_taken = time_single_file_extraction(filepath)
    
    # Hanya tambahkan ke list jika pemrosesan berhasil
    if time_taken is not None:
        extraction_times.append(time_taken)

# Buat DataFrame dari list waktu untuk analisis lebih lanjut
time_df = pd.DataFrame(extraction_times, columns=['processing_time_seconds'])

# Tampilkan 5 catatan waktu pertama
print("\n✅ Pengukuran selesai.")
print("\nCuplikan data waktu pemrosesan (detik per file):")
print(time_df.head())

# Tampilkan statistik dasar
print("\n--- Statistik Waktu Pemrosesan ---")
print(f"File berhasil diukur: {len(time_df)}")
print(f"Waktu Rata-rata: {time_df['processing_time_seconds'].mean():.4f} detik")
print(f"Waktu Median   : {time_df['processing_time_seconds'].median():.4f} detik")
print(f"Total Waktu    : {time_df['processing_time_seconds'].sum():.2f} detik")

d:\Anacondas\envs\newenv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Memulai pengukuran waktu ekstraksi file satu per satu...


100%|██████████| 1000/1000 [00:30<00:00, 32.95it/s]


✅ Pengukuran selesai.

Cuplikan data waktu pemrosesan (detik per file):
   processing_time_seconds
0                 1.449459
1                 0.012849
2                 0.018283
3                 0.014994
4                 0.012158

--- Statistik Waktu Pemrosesan ---
File berhasil diukur: 1000
Waktu Rata-rata: 0.0299 detik
Waktu Median   : 0.0257 detik
Total Waktu    : 29.93 detik


In [ ]:
time_df.to_csv("../data/output/extraction_time_numeric.csv", index=False)

In [ ]:
# Pastikan variabel `data_sample_path` sudah ada dari proses sebelumnya.
# Kode ini mengasumsikan `data_sample_path` adalah DataFrame dengan kolom "filepath".

def time_mel_creation(filepath, sr=16000, n_mels=128, fixed_frames=128):
    """
    Menjalankan proses pembuatan Mel spectrogram untuk satu file
    dan HANYA mengembalikan waktu yang dibutuhkan (dalam detik).
    """
    start_time = time.perf_counter()
    try:
        y, _ = librosa.load(filepath, sr=sr, mono=True)
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
        mel_resized = resize(mel_spectrogram_db, (n_mels, fixed_frames), mode='reflect', anti_aliasing=True)
        # Hasil komputasi tidak disimpan untuk menghemat memori
        _ = (mel_resized + 80) / 80
        elapsed_time = time.perf_counter() - start_time
        return elapsed_time
    except Exception as e:
        return None

# List untuk menampung catatan waktu
processing_times = []

print("Memulai pengukuran waktu pembuatan Mel spectrogram per file...")
# Menggunakan data_sample_path sesuai permintaan
for filepath in tqdm(data_sample_path["filepath"], total=len(data_sample_path)):
    
    time_taken = time_mel_creation(filepath)
    
    if time_taken is not None:
        processing_times.append(time_taken)

# Buat DataFrame dari list waktu
time_df = pd.DataFrame(processing_times, columns=['mel_creation_time_seconds'])

# --- Tampilkan Hasil ---
print("\n✅ Pengukuran selesai.")
print("\nCuplikan data waktu pemrosesan (detik per file):")
print(time_df.head())

print("\n--- Statistik Waktu Pemrosesan ---")
# Menampilkan jumlah file yang berhasil diukur dari total di data_sample_path
print(f"File berhasil diukur: {len(time_df)} dari {len(data_sample_path)}")
print(f"Waktu Rata-rata: {time_df['mel_creation_time_seconds'].mean():.4f} detik")
print(f"Waktu Median   : {time_df['mel_creation_time_seconds'].median():.4f} detik")
print(f"Total Waktu    : {time_df['mel_creation_time_seconds'].sum():.2f} detik")

d:\Anacondas\envs\newenv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Memulai pengukuran waktu pembuatan Mel spectrogram per file...


100%|██████████| 1000/1000 [00:10<00:00, 99.82it/s]


✅ Pengukuran selesai.

Cuplikan data waktu pemrosesan (detik per file):
   mel_creation_time_seconds
0                   1.688614
1                   0.005839
2                   0.005781
3                   0.009606
4                   0.005799

--- Statistik Waktu Pemrosesan ---
File berhasil diukur: 1000 dari 1000
Waktu Rata-rata: 0.0099 detik
Waktu Median   : 0.0078 detik
Total Waktu    : 9.92 detik


In [ ]:
time_df.to_csv("../data/output/extraction_time_CNN.csv")

In [ ]:
# Pastikan variabel `data_sample_path` sudah ada dari proses sebelumnya.
# Kode ini mengasumsikan `data_sample_path` adalah DataFrame dengan kolom "filepath".

def time_segmented_mel_creation(filepath, sr=16000, n_mels=128, fixed_frames=128, timesteps=32):
    """
    Menjalankan proses pembuatan segmented Mel spectrogram untuk satu file
    dan HANYA mengembalikan waktu yang dibutuhkan (dalam detik).
    """
    start_time = time.perf_counter()
    try:
        # 1. Muat audio
        y, _ = librosa.load(filepath, sr=sr, mono=True)

        # 2. Buat Mel spectrogram dan normalisasi
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
        mel_spectrogram_db = (mel_spectrogram_db + 80) / 80
        
        # 3. Ubah ukuran
        resized_mel_spectrogram = resize(mel_spectrogram_db, (n_mels, fixed_frames), mode='reflect', anti_aliasing=True)
        
        # 4. Lakukan segmentasi (langkah ini tetap dijalankan untuk akurasi waktu)
        segment_length = fixed_frames // timesteps
        # Hasil komputasi tidak disimpan ke variabel untuk hemat memori
        _ = [resized_mel_spectrogram[:, i * segment_length:(i + 1) * segment_length] for i in range(timesteps)]
        
        # 5. Hitung waktu yang telah berlalu
        elapsed_time = time.perf_counter() - start_time
        return elapsed_time

    except Exception as e:
        # Jika ada file yang error, waktu tidak dicatat
        # print(f"Error processing {filepath}: {e}")
        return None

# List untuk menampung catatan waktu
processing_times = []

print("Memulai pengukuran waktu pembuatan segmented Mel spectrogram per file...")
# Menggunakan data_sample_path sesuai permintaan
for filepath in tqdm(data_sample_path["filepath"], total=len(data_sample_path)):
    
    # Panggil fungsi untuk mengukur waktu
    time_taken = time_segmented_mel_creation(filepath)
    
    # Hanya tambahkan ke list jika pemrosesan berhasil
    if time_taken is not None:
        processing_times.append(time_taken)

# Buat DataFrame dari list waktu
time_df = pd.DataFrame(processing_times, columns=['segmented_mel_time_seconds'])


# --- Tampilkan Hasil ---
print("\n✅ Pengukuran selesai.")
print("\nCuplikan data waktu pemrosesan (detik per file):")
print(time_df.head())

print("\n--- Statistik Waktu Pemrosesan ---")
print(f"File berhasil diukur: {len(time_df)} dari {len(data_sample_path)}")
print(f"Waktu Rata-rata: {time_df['segmented_mel_time_seconds'].mean():.4f} detik")
print(f"Waktu Median   : {time_df['segmented_mel_time_seconds'].median():.4f} detik")
print(f"Total Waktu    : {time_df['segmented_mel_time_seconds'].sum():.2f} detik")

d:\Anacondas\envs\newenv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Memulai pengukuran waktu pembuatan segmented Mel spectrogram per file...


100%|██████████| 1000/1000 [00:09<00:00, 104.03it/s]


✅ Pengukuran selesai.

Cuplikan data waktu pemrosesan (detik per file):
   segmented_mel_time_seconds
0                    1.437140
1                    0.008843
2                    0.007735
3                    0.008717
4                    0.005479

--- Statistik Waktu Pemrosesan ---
File berhasil diukur: 1000 dari 1000
Waktu Rata-rata: 0.0095 detik
Waktu Median   : 0.0078 detik
Total Waktu    : 9.50 detik


In [ ]:
time_df.to_csv("../data/output/extraction_time_CNN_LSTM.csv")

# Classification

In [ ]:
df_train = pd.read_csv("../data/processed/numeric_train_data.csv")


In [3]:
# Acak urutan baris pada dataset train dan test
# df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
# df_test  = df_test.sample(frac=1, random_state=42).reset_index(drop=True)

df_train = df_train
X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]



In [4]:
data_sample = df_train.sample(1000, random_state=42)
X_sample = data_sample.iloc[:, :-1]
y_sample = data_sample.iloc[:, -1]

### Random Forest

In [5]:
feature_rf = [
    'chroma_1', 'chroma_2', 'chroma_3', 'chroma_4', 'chroma_8', 'chroma_9',
    'chroma_11', 'chroma_12', 'spectral_bandwidth_mean',
    'spectral_rolloff_mean', 'mfcc_1', 'mfcc_3', 'mfcc_4', 'mfcc_5',
    'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12',
    'mfcc_13', 'mfcc_14', 'mfcc_15', 'mfcc_16', 'mfcc_17', 'mfcc_18',
    'mfcc_19', 'mfcc_20'
]


X_sample_rf = X_sample[feature_rf]
X_train_rf = X_train[feature_rf]

In [ ]:
params = {
    'n_estimators': 111,
    'max_depth': 15,
    'min_samples_split': 4,
    'min_samples_leaf': 2,
    'max_features': 'log2',
    'class_weight': None,
    'random_state' : 42
}

rf_model = RandomForestClassifier(**params)

# Latih model pada data training
rf_model.fit(X_train_rf, y_train)




RandomForestClassifier(max_depth=15, max_features='log2', min_samples_leaf=2,
                       min_samples_split=4, n_estimators=111, random_state=42)

In [ ]:
# Inisialisasi list untuk menyimpan hasil waktu prediksi per sampel
prediction_times_rf = []

# Loop untuk menghitung waktu prediksi satu per satu untuk setiap sampel
for i in range(1, len(X_sample_rf) + 1):
    print(f"🔁 Iterasi ke-{i}")

    start_time = time.perf_counter()
    _ = rf_model.predict(X_sample_rf.iloc[i-1:i, :])
    elapsed_time = time.perf_counter() - start_time
    time_per_sample = elapsed_time  # hanya satu sampel

    prediction_times_rf.append({
        "Iteration": i,
        "Total_Prediction_Time_sec": elapsed_time,
        "Time_per_Sample_sec": time_per_sample
    })

# Simpan hasil ke DataFrame
df_times_rf = pd.DataFrame(prediction_times_rf)

# Simpan ke CSV
df_times_rf.to_csv("../data/output/rf_prediction_per_sample.csv", index=False)

print("\n✅ Waktu prediksi per sampel berhasil disimpan ke 'rf_prediction_per_sample.csv'")
print("Rata-rata waktu prediksi per sampel:", df_times_rf['Time_per_Sample_sec'].mean(), "detik")


🔁 Iterasi ke-1
🔁 Iterasi ke-2
🔁 Iterasi ke-3
🔁 Iterasi ke-4
🔁 Iterasi ke-5
🔁 Iterasi ke-6
🔁 Iterasi ke-7
🔁 Iterasi ke-8
🔁 Iterasi ke-9
🔁 Iterasi ke-10
🔁 Iterasi ke-11
🔁 Iterasi ke-12
🔁 Iterasi ke-13
🔁 Iterasi ke-14
🔁 Iterasi ke-15
🔁 Iterasi ke-16
🔁 Iterasi ke-17
🔁 Iterasi ke-18
🔁 Iterasi ke-19
🔁 Iterasi ke-20
🔁 Iterasi ke-21
🔁 Iterasi ke-22
🔁 Iterasi ke-23
🔁 Iterasi ke-24
🔁 Iterasi ke-25
🔁 Iterasi ke-26
🔁 Iterasi ke-27
🔁 Iterasi ke-28
🔁 Iterasi ke-29
🔁 Iterasi ke-30
🔁 Iterasi ke-31
🔁 Iterasi ke-32
🔁 Iterasi ke-33
🔁 Iterasi ke-34
🔁 Iterasi ke-35
🔁 Iterasi ke-36
🔁 Iterasi ke-37
🔁 Iterasi ke-38
🔁 Iterasi ke-39
🔁 Iterasi ke-40
🔁 Iterasi ke-41
🔁 Iterasi ke-42
🔁 Iterasi ke-43
🔁 Iterasi ke-44
🔁 Iterasi ke-45
🔁 Iterasi ke-46
🔁 Iterasi ke-47
🔁 Iterasi ke-48
🔁 Iterasi ke-49
🔁 Iterasi ke-50
🔁 Iterasi ke-51
🔁 Iterasi ke-52
🔁 Iterasi ke-53
🔁 Iterasi ke-54
🔁 Iterasi ke-55
🔁 Iterasi ke-56
🔁 Iterasi ke-57
🔁 Iterasi ke-58
🔁 Iterasi ke-59
🔁 Iterasi ke-60
🔁 Iterasi ke-61
🔁 Iterasi ke-62
🔁 Iterasi ke-63
🔁

In [ ]:
import gc
del rf_model
gc.collect()


544

### SVM

In [9]:
feature_svm = ['chroma_1', 'chroma_4', 'chroma_7', 'chroma_8', 'chroma_11', 'rms_mean',
       'spectral_centroid_mean', 'spectral_bandwidth_mean',
       'spectral_rolloff_mean', 'zcr_mean', 'mfcc_1', 'mfcc_2', 'mfcc_3',
       'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_9', 'mfcc_10', 'mfcc_14',
       'mfcc_17', 'mfcc_18', 'mfcc_19', 'mfcc_20']

X_train_svm = X_train[feature_svm]
X_sample_svm = X_sample[feature_svm]

In [10]:
# Inisialisasi model SVM dengan hyperparameter terbaik
params = {
    'C': 19.519063724517864,
    'gamma': 0.4791572427747828,
    'class_weight': None,
    'random_state' :42
}

svm_model = SVC(**params)

# Latih model pada data training
svm_model.fit(X_train_svm, y_train)


SVC(C=19.519063724517864, gamma=0.4791572427747828, random_state=42)

In [ ]:
# Inisialisasi list untuk menyimpan hasil waktu prediksi per sampel
prediction_times_svm = []

# Loop untuk menghitung waktu prediksi per 1 sampel
for i in range(1, len(X_sample_svm) + 1):
    print(f"🔁 Iterasi ke-{i}")

    start_time = time.perf_counter()
    _ = svm_model.predict(X_sample_svm.iloc[i-1:i, :])
    elapsed_time = time.perf_counter() - start_time
    time_per_sample = elapsed_time  # satu sampel, jadi waktu per sampel = total waktu

    prediction_times_svm.append({
        "Iteration": i,
        "Total_Prediction_Time_sec": elapsed_time,
        "Time_per_Sample_sec": time_per_sample
    })

# Simpan hasil ke DataFrame
df_times_svm = pd.DataFrame(prediction_times_svm)

# Simpan ke CSV
df_times_svm.to_csv("../data/output/svm_prediction_per_sample.csv", index=False)

print("✅ Waktu prediksi per sampel berhasil disimpan ke 'svm_prediction_per_sample.csv'")
print("Rata-rata waktu prediksi per sampel:", df_times_svm['Time_per_Sample_sec'].mean(), "detik")


🔁 Iterasi ke-1
🔁 Iterasi ke-2
🔁 Iterasi ke-3
🔁 Iterasi ke-4
🔁 Iterasi ke-5
🔁 Iterasi ke-6
🔁 Iterasi ke-7
🔁 Iterasi ke-8
🔁 Iterasi ke-9
🔁 Iterasi ke-10
🔁 Iterasi ke-11
🔁 Iterasi ke-12
🔁 Iterasi ke-13
🔁 Iterasi ke-14
🔁 Iterasi ke-15
🔁 Iterasi ke-16
🔁 Iterasi ke-17
🔁 Iterasi ke-18
🔁 Iterasi ke-19
🔁 Iterasi ke-20
🔁 Iterasi ke-21
🔁 Iterasi ke-22
🔁 Iterasi ke-23
🔁 Iterasi ke-24
🔁 Iterasi ke-25
🔁 Iterasi ke-26
🔁 Iterasi ke-27
🔁 Iterasi ke-28
🔁 Iterasi ke-29
🔁 Iterasi ke-30
🔁 Iterasi ke-31
🔁 Iterasi ke-32
🔁 Iterasi ke-33
🔁 Iterasi ke-34
🔁 Iterasi ke-35
🔁 Iterasi ke-36
🔁 Iterasi ke-37
🔁 Iterasi ke-38
🔁 Iterasi ke-39
🔁 Iterasi ke-40
🔁 Iterasi ke-41
🔁 Iterasi ke-42
🔁 Iterasi ke-43
🔁 Iterasi ke-44
🔁 Iterasi ke-45
🔁 Iterasi ke-46
🔁 Iterasi ke-47
🔁 Iterasi ke-48
🔁 Iterasi ke-49
🔁 Iterasi ke-50
🔁 Iterasi ke-51
🔁 Iterasi ke-52
🔁 Iterasi ke-53
🔁 Iterasi ke-54
🔁 Iterasi ke-55
🔁 Iterasi ke-56
🔁 Iterasi ke-57
🔁 Iterasi ke-58
🔁 Iterasi ke-59
🔁 Iterasi ke-60
🔁 Iterasi ke-61
🔁 Iterasi ke-62
🔁 Iterasi ke-63
🔁

Model CNN

In [ ]:
data_val_cnn = np.load("../data/processed/cnn_train_data.npz")
X = data_val_cnn["data"]

# number of samples in X
n_samples = X.shape[0]

# set random seed for reproducibility
rng = np.random.default_rng(42)

# sample 1000 random indices without replacement
indices = rng.choice(n_samples, size=1000, replace=False)

# select those rows
X_sample_cnn = X[indices]


In [ ]:
# Ganti 'model_name.h5' dengan path file model .h5 Anda
model_cnn = load_model('../model/model_cnn_fix.h5')



In [ ]:
# Inisialisasi list untuk menyimpan hasil waktu prediksi per sampel
prediction_times_cnn = []

# Loop untuk menghitung waktu prediksi satu per satu untuk setiap sampel
for i in range(1, len(X_sample_cnn) + 1):
    print(f"🔁 Iterasi ke-{i}")

    start_time = time.perf_counter()
    _ = model_cnn.predict(X_sample_cnn[i-1:i], verbose = 0)  # pastikan input shape sesuai (misalnya [1, height, width, channels])
    elapsed_time = time.perf_counter() - start_time
    time_per_sample = elapsed_time  # hanya satu sampel

    prediction_times_cnn.append({
        "Iteration": i,
        "Total_Prediction_Time_sec": elapsed_time,
        "Time_per_Sample_sec": time_per_sample
    })

# Simpan hasil ke DataFrame
df_times_cnn = pd.DataFrame(prediction_times_cnn)

# Simpan ke CSV
df_times_cnn.to_csv("../data/output/cnn_prediction_per_sample.csv", index=False)

print("\n✅ Waktu prediksi per sampel berhasil disimpan ke 'cnn_prediction_per_sample.csv'")
print("Rata-rata waktu prediksi per sampel:", df_times_cnn['Time_per_Sample_sec'].mean(), "detik")


🔁 Iterasi ke-1
🔁 Iterasi ke-2
🔁 Iterasi ke-3
🔁 Iterasi ke-4
🔁 Iterasi ke-5
🔁 Iterasi ke-6
🔁 Iterasi ke-7
🔁 Iterasi ke-8
🔁 Iterasi ke-9
🔁 Iterasi ke-10
🔁 Iterasi ke-11
🔁 Iterasi ke-12
🔁 Iterasi ke-13
🔁 Iterasi ke-14
🔁 Iterasi ke-15
🔁 Iterasi ke-16
🔁 Iterasi ke-17
🔁 Iterasi ke-18
🔁 Iterasi ke-19
🔁 Iterasi ke-20
🔁 Iterasi ke-21
🔁 Iterasi ke-22
🔁 Iterasi ke-23
🔁 Iterasi ke-24
🔁 Iterasi ke-25
🔁 Iterasi ke-26
🔁 Iterasi ke-27
🔁 Iterasi ke-28
🔁 Iterasi ke-29
🔁 Iterasi ke-30
🔁 Iterasi ke-31
🔁 Iterasi ke-32
🔁 Iterasi ke-33
🔁 Iterasi ke-34
🔁 Iterasi ke-35
🔁 Iterasi ke-36
🔁 Iterasi ke-37
🔁 Iterasi ke-38
🔁 Iterasi ke-39
🔁 Iterasi ke-40
🔁 Iterasi ke-41
🔁 Iterasi ke-42
🔁 Iterasi ke-43
🔁 Iterasi ke-44
🔁 Iterasi ke-45
🔁 Iterasi ke-46
🔁 Iterasi ke-47
🔁 Iterasi ke-48
🔁 Iterasi ke-49
🔁 Iterasi ke-50
🔁 Iterasi ke-51
🔁 Iterasi ke-52
🔁 Iterasi ke-53
🔁 Iterasi ke-54
🔁 Iterasi ke-55
🔁 Iterasi ke-56
🔁 Iterasi ke-57
🔁 Iterasi ke-58
🔁 Iterasi ke-59
🔁 Iterasi ke-60
🔁 Iterasi ke-61
🔁 Iterasi ke-62
🔁 Iterasi ke-63
🔁

In [10]:
import gc
del model_cnn
gc.collect()

9838

Model CNN-LSTM

In [ ]:
data_val_cnn_lstm = np.load("../data/processed/cnn_lstm_train_data.npz")
X = data_val_cnn_lstm["data"]

n_samples = X.shape[0]

rng = np.random.default_rng(42)  # random generator with seed for reproducibility
indices = rng.choice(n_samples, size=1000, replace=False)

X_sample_cnn_lstm = X[indices]


In [ ]:
# Ganti 'model_name.h5' dengan path file model .h5 Anda
model_cnn_lstm = load_model('../model/model_cnn_lstm_fix.h5')

In [ ]:
# Inisialisasi list untuk menyimpan hasil waktu prediksi per sampel
prediction_times_cnn_lstm = []

# Loop untuk menghitung waktu prediksi satu per satu untuk setiap sampel
for i in range(1, len(X_sample_cnn_lstm) + 1):
    print(f"🔁 Iterasi ke-{i}")

    start_time = time.perf_counter()
    _ = model_cnn_lstm.predict(X_sample_cnn_lstm[i-1:i], verbose=0)  # asumsi input batch shape sudah sesuai
    elapsed_time = time.perf_counter() - start_time
    time_per_sample = elapsed_time  # hanya satu sampel

    prediction_times_cnn_lstm.append({
        "Iteration": i,
        "Total_Prediction_Time_sec": elapsed_time,
        "Time_per_Sample_sec": time_per_sample
    })

# Simpan hasil ke DataFrame
df_times_cnn_lstm = pd.DataFrame(prediction_times_cnn_lstm)

# Simpan ke CSV
df_times_cnn_lstm.to_csv("../data/output/cnn_lstm_prediction_per_sample.csv", index=False)

print("\n✅ Waktu prediksi per sampel berhasil disimpan ke 'cnn_lstm_prediction_per_sample.csv'")
print("Rata-rata waktu prediksi per sampel:", df_times_cnn_lstm['Time_per_Sample_sec'].mean(), "detik")


🔁 Iterasi ke-1
🔁 Iterasi ke-2
🔁 Iterasi ke-3
🔁 Iterasi ke-4
🔁 Iterasi ke-5
🔁 Iterasi ke-6
🔁 Iterasi ke-7
🔁 Iterasi ke-8
🔁 Iterasi ke-9
🔁 Iterasi ke-10
🔁 Iterasi ke-11
🔁 Iterasi ke-12
🔁 Iterasi ke-13
🔁 Iterasi ke-14
🔁 Iterasi ke-15
🔁 Iterasi ke-16
🔁 Iterasi ke-17
🔁 Iterasi ke-18
🔁 Iterasi ke-19
🔁 Iterasi ke-20
🔁 Iterasi ke-21
🔁 Iterasi ke-22
🔁 Iterasi ke-23
🔁 Iterasi ke-24
🔁 Iterasi ke-25
🔁 Iterasi ke-26
🔁 Iterasi ke-27
🔁 Iterasi ke-28
🔁 Iterasi ke-29
🔁 Iterasi ke-30
🔁 Iterasi ke-31
🔁 Iterasi ke-32
🔁 Iterasi ke-33
🔁 Iterasi ke-34
🔁 Iterasi ke-35
🔁 Iterasi ke-36
🔁 Iterasi ke-37
🔁 Iterasi ke-38
🔁 Iterasi ke-39
🔁 Iterasi ke-40
🔁 Iterasi ke-41
🔁 Iterasi ke-42
🔁 Iterasi ke-43
🔁 Iterasi ke-44
🔁 Iterasi ke-45
🔁 Iterasi ke-46
🔁 Iterasi ke-47
🔁 Iterasi ke-48
🔁 Iterasi ke-49
🔁 Iterasi ke-50
🔁 Iterasi ke-51
🔁 Iterasi ke-52
🔁 Iterasi ke-53
🔁 Iterasi ke-54
🔁 Iterasi ke-55
🔁 Iterasi ke-56
🔁 Iterasi ke-57
🔁 Iterasi ke-58
🔁 Iterasi ke-59
🔁 Iterasi ke-60
🔁 Iterasi ke-61
🔁 Iterasi ke-62
🔁 Iterasi ke-63
🔁

### Perhitungan waktu pemodelan

In [ ]:
df_train = pd.read_csv("../data/processed/numeric_train_data.csv")
df_val = pd.read_csv("../data/processed/numeric_val_data.csv")
df_test = pd.read_csv("../data/processed/numeric_test_data.csv")

In [18]:
df_train_final = pd.concat([df_train,df_val], axis=0)
df_train_final

,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,chroma_6,chroma_7,chroma_8,chroma_9,chroma_10,...,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,label
0,0.504315,0.548461,0.501602,0.542981,0.519444,0.689757,0.674948,0.571628,0.550304,0.521429,...,0.580840,0.591705,0.399666,0.599349,0.472456,0.647290,0.416770,0.721205,0.244970,0
1,0.433211,0.484077,0.449849,0.491398,0.503308,0.682860,0.695454,0.762369,0.770101,0.703590,...,0.477089,0.576566,0.364740,0.567235,0.376243,0.694686,0.574954,0.745847,0.375365,0
2,0.650005,0.730896,0.709415,0.698076,0.638344,0.693965,0.629554,0.687421,0.722584,0.719230,...,0.557636,0.580289,0.398246,0.782590,0.566742,0.696631,0.526107,0.836288,0.485992,0
3,0.515072,0.483673,0.467737,0.631756,0.627723,0.595867,0.515102,0.535442,0.546995,0.577641,...,0.404663,0.627310,0.343542,0.660205,0.411806,0.616249,0.412144,0.656624,0.200678,0
4,0.302506,0.493074,0.504830,0.383060,0.388507,0.704446,0.840085,0.752893,0.431209,0.273619,...,0.507439,0.545278,0.397998,0.664384,0.438337,0.768158,0.265497,0.659639,0.276219,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7637,0.437047,0.555612,0.487570,0.609676,0.565383,0.621709,0.705636,0.768562,0.671040,0.537779,...,0.625622,0.357911,0.573465,0.500598,0.638202,0.485155,0.570285,0.447402,0.553567,1
7638,0.286778,0.296154,0.394823,0.698401,0.685904,0.722048,0.569149,0.679403,0.785093,0.608060,...,0.631064,0.517908,0.599596,0.580383,0.628368,0.310323,0.410315,0.638640,0.454653,1
7639,0.386399,0.561362,0.596431,0.696707,0.591178,0.571610,0.670914,0.818815,0.770197,0.577832,...,0.660530,0.477367,0.563112,0.509746,0.566321,0.331353,0.613877,0.637037,0.532293,1
7640,0.441040,0.602926,0.523737,0.542881,0.670638,0.679416,0.491501,0.493725,0.556791,0.813018,...,0.529096,0.396013,0.642967,0.527645,0.665692,0.178644,0.453539,0.477542,0.553312,1


In [20]:
# === Data Splits ===
X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]

X_train_final = df_train_final.iloc[:, :-1]
y_train_final = df_train_final.iloc[:, -1]

X_val = df_val.iloc[:, :-1]
y_val = df_val.iloc[:, -1]

X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]

In [22]:
feature_rf = [
    'chroma_1', 'chroma_2', 'chroma_3', 'chroma_4', 'chroma_8', 'chroma_9',
    'chroma_11', 'chroma_12', 'spectral_bandwidth_mean',
    'spectral_rolloff_mean', 'mfcc_1', 'mfcc_3', 'mfcc_4', 'mfcc_5',
    'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12',
    'mfcc_13', 'mfcc_14', 'mfcc_15', 'mfcc_16', 'mfcc_17', 'mfcc_18',
    'mfcc_19', 'mfcc_20'
]

X_train_rf = X_train_final[feature_rf]
X_val_rf = X_val[feature_rf]
X_test_rf = X_test[feature_rf]

In [ ]:

# === Random Forest Hyperparameters ===
params = {
    'n_estimators': 111,
    'max_depth': 15,
    'min_samples_split': 4,
    'min_samples_leaf': 2,
    'max_features': 'log2',
    'class_weight': None,
    'random_state' : 42
}

# === Model Training ===
start_time = time.perf_counter()
rf_model = RandomForestClassifier(**params)
rf_model.fit(X_train_rf, y_train_final)  # Training model
elapsed_time = time.perf_counter() - start_time

print(f"Training time: {elapsed_time} seconds")


Training time: 5.991392699999778 seconds


In [25]:
import gc
del rf_model
gc.collect()

3122

SVM

In [ ]:
df_train_final = pd.concat([df_train,df_val], axis=0)
df_train_final

In [ ]:
# === Data Splits ===
X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]

X_train_final = df_train_final.iloc[:, :-1]
y_train_final = df_train_final.iloc[:, -1]

X_val = df_val.iloc[:, :-1]
y_val = df_val.iloc[:, -1]

X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]

In [27]:
feature_svm = ['chroma_1', 'chroma_4', 'chroma_7', 'chroma_8', 'chroma_11', 'rms_mean',
       'spectral_centroid_mean', 'spectral_bandwidth_mean',
       'spectral_rolloff_mean', 'zcr_mean', 'mfcc_1', 'mfcc_2', 'mfcc_3',
       'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_9', 'mfcc_10', 'mfcc_14',
       'mfcc_17', 'mfcc_18', 'mfcc_19', 'mfcc_20']

X_train_svm = X_train_final[feature_svm]
X_val_svm = X_val[feature_svm]
X_test_svm = X_test[feature_svm]

In [ ]:


# Inisialisasi model SVM dengan hyperparameter terbaik
params = {
    'C': 19.519063724517864,
    'gamma': 0.4791572427747828,
    'class_weight': None,
    'random_state' :42
}


# === Model Training ===
start_time = time.perf_counter()
svm_model = SVC(**params)
svm_model.fit(X_train_svm, y_train_final)
elapsed_time = time.perf_counter() - start_time

print(f"Training time: {elapsed_time} seconds")


Training time: 2.148612899998625 seconds


In [ ]:
import gc
del svm_model
gc.collect()

CNN

In [ ]:
# Load kedua file .npz
data_train_cnn = np.load("../data/processed/cnn_train_data.npz")
data_val_cnn = np.load("../data/processed/cnn_val_data.npz")
data_test_cnn = np.load("../data/processed/cnn_test_data.npz")

In [4]:
X_train_cnn = data_train_cnn["data"]
y_train_cnn = data_train_cnn["label"]
X_val_cnn = data_val_cnn["data"]
y_val_cnn = data_val_cnn["label"]
X_test_cnn = data_test_cnn["data"]
y_test_cnn = data_test_cnn["label"]

In [5]:
X_train_cnn = np.concatenate([data_train_cnn['data'], data_val_cnn['data']], axis=0)
y_train_cnn = np.concatenate([data_train_cnn['label'], data_val_cnn['label']], axis=0)

In [6]:
# Lightweight CNN Model
model_cnn = Sequential([
    Conv2D(8, (3, 3), activation='relu', input_shape=(128, 128, 1), name='conv2d_1'),
    BatchNormalization(),
    MaxPooling2D(2, 2, name='maxpool_1'),

    Conv2D(16, (3, 3), activation='relu', name='conv2d_2'),
    MaxPooling2D(2, 2, name='maxpool_2'),

    Flatten(name='flatten'),
    Dense(32, activation='relu', name='dense_1'),
    Dropout(0.5, name='dropout_1'),

    Dense(1, activation='sigmoid', name='output')  # Binary classification
])

# Compile the model
model_cnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)



In [7]:
# Training model dengan data validasi eksplisit
start_time = time.perf_counter()
history_cnn = model_cnn.fit(
    X_train_cnn,
    y_train_cnn,
    epochs=30,
    batch_size=32
)
elapsed_time = time.perf_counter() - start_time
print(f"Training time: {elapsed_time} seconds")


Epoch 1/30
481/481 [==============================] - 47s 94ms/step - loss: 0.4003 - accuracy: 0.8231
Epoch 2/30
481/481 [==============================] - 43s 90ms/step - loss: 0.2816 - accuracy: 0.9038
Epoch 3/30
481/481 [==============================] - 41s 86ms/step - loss: 0.2339 - accuracy: 0.9217
Epoch 4/30
481/481 [==============================] - 42s 86ms/step - loss: 0.1981 - accuracy: 0.9374
Epoch 5/30
481/481 [==============================] - 42s 86ms/step - loss: 0.1795 - accuracy: 0.9441
Epoch 6/30
481/481 [==============================] - 42s 87ms/step - loss: 0.1627 - accuracy: 0.9496
Epoch 7/30
481/481 [==============================] - 40s 84ms/step - loss: 0.1438 - accuracy: 0.9569
Epoch 8/30
481/481 [==============================] - 40s 84ms/step - loss: 0.1309 - accuracy: 0.9607
Epoch 9/30
481/481 [==============================] - 42s 87ms/step - loss: 0.1216 - accuracy: 0.9622
Epoch 10/30
481/481 [==============================] - 37s 77ms/step - loss: 0.120

In [9]:
import gc
del model_cnn
gc.collect()

1613

CNN-LSTM

In [ ]:
# Load kedua file .npz
data_train_cnn_lstm = np.load("../data/processed/cnn_lstm_train_data.npz")
data_val_cnn_lstm = np.load("../data/processed/cnn_lstm_val_data.npz")
data_test_cnn_lstm = np.load("../data/processed/cnn_lstm_test_data.npz")

In [9]:
X_train_cnn_lstm = data_train_cnn_lstm["data"]
y_train_cnn_lstm = data_train_cnn_lstm["label"]
X_val_cnn_lstm = data_val_cnn_lstm["data"]
y_val_cnn_lstm = data_val_cnn_lstm["label"]
X_test_cnn_lstm = data_test_cnn_lstm["data"]
y_test_cnn_lstm = data_test_cnn_lstm["label"]

In [10]:
X_train_cnn_lstm = np.concatenate([data_train_cnn_lstm['data'], data_val_cnn_lstm['data']], axis=0)
y_train_cnn_lstm = np.concatenate([data_train_cnn_lstm['label'], data_val_cnn_lstm['label']], axis=0)

In [11]:
# Lightweight CNN-LSTM model
model_cnn_lstm = Sequential([
    # TimeDistributed CNN layers
    TimeDistributed(Conv2D(8, (3, 3), activation='relu', padding='same'),
                    input_shape=(32, 128, 4, 1), name="conv2d_1"),
    TimeDistributed(BatchNormalization(), name="batchnorm_1"),
    TimeDistributed(MaxPooling2D((2, 2), padding="same"), name="maxpool_1"),

    TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same'), name="conv2d_2"),
    TimeDistributed(MaxPooling2D((2, 2), padding="same"), name="maxpool_2"),

    TimeDistributed(Flatten(), name="flatten"),

    # LSTM layers
    LSTM(64, return_sequences=True, name="lstm_1"),
    LSTM(32, return_sequences=False, name="lstm_2"),

    # Fully connected layers
    Dense(32, activation='relu', name="dense_1"),
    Dropout(0.4, name="dropout_1"),

    Dense(1, activation='sigmoid', name="output")
])

# Compile the model
model_cnn_lstm.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)



# Summary
model_cnn_lstm.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (TimeDistributed)  (None, 32, 128, 4, 8)     80        
                                                                 
 batchnorm_1 (TimeDistribute  (None, 32, 128, 4, 8)    32        
 d)                                                              
                                                                 
 maxpool_1 (TimeDistributed)  (None, 32, 64, 2, 8)     0         
                                                                 
 conv2d_2 (TimeDistributed)  (None, 32, 64, 2, 16)     1168      
                                                                 
 maxpool_2 (TimeDistributed)  (None, 32, 32, 1, 16)    0         
                                                                 
 flatten (TimeDistributed)   (None, 32, 512)           0         
                                                        

In [12]:
# Training model dengan data validasi eksplisit
start_time = time.perf_counter()
history_cnn_lstm = model_cnn_lstm.fit(
    X_train_cnn_lstm,
    y_train_cnn_lstm,
    epochs=30,
    batch_size=32
)
elapsed_time = time.perf_counter() - start_time
print(f"Training time: {elapsed_time} seconds")


Epoch 1/30
481/481 [==============================] - 55s 103ms/step - loss: 0.3939 - accuracy: 0.8023
Epoch 2/30
481/481 [==============================] - 52s 108ms/step - loss: 0.1945 - accuracy: 0.9233
Epoch 3/30
481/481 [==============================] - 53s 110ms/step - loss: 0.0679 - accuracy: 0.9774
Epoch 4/30
481/481 [==============================] - 53s 110ms/step - loss: 0.0159 - accuracy: 0.9956
Epoch 5/30
481/481 [==============================] - 53s 110ms/step - loss: 0.0093 - accuracy: 0.9975
Epoch 6/30
481/481 [==============================] - 50s 104ms/step - loss: 0.0022 - accuracy: 0.9995
Epoch 7/30
481/481 [==============================] - 50s 105ms/step - loss: 0.0137 - accuracy: 0.9969
Epoch 8/30
481/481 [==============================] - 53s 111ms/step - loss: 0.0142 - accuracy: 0.9962
Epoch 9/30
481/481 [==============================] - 53s 109ms/step - loss: 0.0056 - accuracy: 0.9988
Epoch 10/30
481/481 [==============================] - 55s 113ms/step - l